# Storm Research Assistant

Reference
https://github.com/langchain-ai/langgraph/blob/main/examples/storm/storm.ipynb


In [1]:
from storm import *
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic


fast_llm = ChatOpenAI(model="gpt-3.5-turbo")
# long_context_llm = ChatOpenAI(model="gpt-4-turbo-preview")
long_context_llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# haiku model
haiku_model_name = "claude-3-haiku-20240307"
# fast_llm = ChatAnthropic(model_name=haiku_model_name)
long_context_llm = ChatAnthropic(model_name=haiku_model_name)


embeddings = get_gpt4all_embeddings()

vectorstore_dir = "./data/storm/vectorstore/"
vectorstore = Chroma(persist_directory=vectorstore_dir, embedding_function=embeddings)

interview_config = InterviewConfig(long_llm=long_context_llm, 
                                   fast_llm=fast_llm, 
                                   max_conversations=3, 
                                   max_reference_length=10000,
                                   tags_to_extract=[ "p", "h1", "h2", "h3"],
                                   embeddings=embeddings,
                                   vectorstore=vectorstore,
                                   vectorstore_dir=vectorstore_dir,
                                   runnable_config=RunnableConfig()
                                   )

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [2]:
example_topic = "Increase development productivity by using Docker compose and local docker labs"

## Test Single Interview using StormInterviewGraph1


In [3]:

# interview_graph = StormInterviewGraph1(interview_config=interview_config)

# context = "increase development productivity by using Docker compose and local docker labs"

# tm1 = HumanMessage(content="What is the best way to increase development productivity by using Docker compose and local docker labs", name="John Smith")

# test_state = InterviewState(
#     context=context,
#     interview_config=interview_config,
#     editor=Editor(affiliation="Software Company X", name="John Doe", role="Lead Software Engineer", description="Experienced software engineer."),
#     messages=[],
#     references={}
# )

# # Run interview

# final_step = None
# async for step in interview_graph.graph.astream(test_state.as_dict()):
#     name = next(iter(step))
#     print(name)
#     print(f"Processing step: {name}")
#     print("-- ", str(step[name]["messages"])[:300])
#     if END in step:
#         final_step = step
        
# final_state = next(iter(final_step.values()))
# final_state

## Test Full Graph


In [4]:

i = Interviews(topic=example_topic, interview_config=interview_config)

g = StormGraph(interview_config=interview_config, topic=example_topic)
g1 = await g.graph.ainvoke(i.as_dict())

g1x = Interviews.from_dict(g1)
g1x.outline


-- Survey Subjects for Topic: [Increase development productivity by using Docker compose and local docker labs] --

Initial Outline: Increase Development Productivity with Docker Compose and Local Docker Labs - [Section(section_title='Introduction to Docker Compose', description='Explanation of Docker Compose and its role in simplifying multi-container Docker applications.', subsections=[Subsection(subsection_title='What is Docker Compose?', description='Definition and basic usage of Docker Compose.'), Subsection(subsection_title='Advantages of Docker Compose', description='Benefits of using Docker Compose for development and deployment.')]), Section(section_title='Setting Up Local Docker Labs', description='Steps to create and manage local Docker labs for increased productivity.', subsections=[Subsection(subsection_title='Creating a Local Docker Lab', description='Guidelines on setting up a local Docker lab environment.'), Subsection(subsection_title='Best Practices for Local Docker 

/Users/tapiwamaruni/anaconda3/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/tapiwamaruni/anaconda3/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Retrieved 7 wiki batches for Topic: Increase development productivity by using Docker compose and local docker labs:

	Retrieved doc: Docker (software) - https://en.wikipedia.org/wiki/Docker_(software)
	Retrieved doc: Docker, Inc. - https://en.wikipedia.org/wiki/Docker,_Inc.
	Retrieved doc: Docker (software) - https://en.wikipedia.org/wiki/Docker_(software)
	Retrieved doc: Buildkite - https://en.wikipedia.org/wiki/Buildkite
	Retrieved doc: TiDB - https://en.wikipedia.org/wiki/TiDB
	Retrieved doc: Containerization - https://en.wikipedia.org/wiki/Containerization
	Retrieved doc: Containerization (computing) - https://en.wikipedia.org/wiki/Containerization_(computing)
	Retrieved doc: Containerized housing unit - https://en.wikipedia.org/wiki/Containerized_housing_unit
	Retrieved doc: DevOps - https://en.wikipedia.org/wiki/DevOps
	Retrieved doc: Azure DevOps Server - https://en.wikipedia.org/wiki/Azure_DevOps_Server
	Retrieved doc: DevOps toolchain - https://en.wikipedia.org/wiki/DevOps_to

2024-04-27 13:20:40,715 [MainThread  ] [INFO ]  Search result: {'ids': ['e2d879d1-3655-4da0-bd94-4f5f24de1ff4', 'eb7060fa-49c0-4f02-8043-033923a98793'], 'embeddings': None, 'metadatas': [{'categories': "['2013 software', 'Articles with short description', 'Containerization software', 'Free software programmed in Go', 'Free virtualization software', 'Linux containerization', 'Operating system security', 'Operating system technology', 'Short description is different from Wikidata', 'Software using the Apache license', 'Use mdy dates from September 2018', 'Virtualization software for Linux']", 'image_urls': "['https://upload.wikimedia.org/wikipedia/commons/0/09/Docker-linux-interfaces.svg', 'https://upload.wikimedia.org/wikipedia/commons/4/4e/Docker_%28container_engine%29_logo.svg', 'https://upload.wikimedia.org/wikipedia/commons/b/b0/NewTux.svg', 'https://upload.wikimedia.org/wikipedia/en/f/f4/Docker_logo.svg', 'https://upload.wikimedia.org/wikipedia/en/8/8a/OOjs_UI_icon_edit-ltr-progres

Generated 2 perspectives for Topic: [Increase development productivity by using Docker compose and local docker labs]

>> Generated perspective for: John Doe 
Affiliation: - University of ABC
Persona: - Name: John Doe
Role: Lead Editor
Affiliation: University of ABC
Description: Specializes in environmental science.

Topic: - Increase development productivity by using Docker compose and local docker labs

>> Generated perspective for: Jane Smith 
Affiliation: - Company XYZ
Persona: - Name: Jane Smith
Role: Editor
Affiliation: Company XYZ
Description: Focuses on technology trends.

Topic: - Increase development productivity by using Docker compose and local docker labs

Handling conversation:
	InterviewState(context='Increase development productivity by using Docker compose and local docker labs', interview_config=InterviewConfig(long_llm=ChatAnthropic(model='claude-3-haiku-20240307', anthropic_api_key=SecretStr('**********'), _client=<anthropic.Anthropic object at 0x116836680>, _async_

...

2024-04-27 13:21:31,853 [MainThread  ] [INFO ]  
Generated section: Increase Development Productivity with Docker Compose and Local Docker Labs 
## Increase Development Productivity with Docker Compose and Local Docker Labs

Docker Compose is a tool that simplifies the process of defining and running multi-container Docker applications. By using Docker Compose, developers can streamline their development workflows, improve consistency across teams, and increase overall productivity.

One of the key advantages of Docker Compose is its ability to ensure that all team members use the same development environment, reducing the "it works on my machine" issue. Docker Compose also provides isolation for each service, running them in their own containers and avoiding conflicts between different components.

To further boost development productivity, organizations can set up local Docker labs, which provide a controlled environment for practical experimentation and testing. By creating a local 

('### Introduction to Docker Compose\n'
 '\n'
 'Docker Compose is a tool that simplifies the process of defining and running '
 'multi-container Docker applications. It allows developers to easily manage '
 'the configuration and deployment of complex, multi-service applications '
 'using a single configuration file. [1]\n'
 '\n'
 '### What is Docker Compose?\n'
 '\n'
 'Docker Compose is a tool that allows you to define and run multi-container '
 'Docker applications. It provides a simple and consistent way to manage the '
 "lifecycle of your application's services, including building, starting, and "
 'stopping them. [2]\n'
 '\n'
 '### Advantages of Docker Compose\n'
 '\n'
 'Using Docker Compose offers several advantages for development, testing, and '
 'deployment processes:\n'
 '- Consistency: Docker Compose ensures that all team members use the same '
 'development environment, reducing "it works on my machine" issues. [2]\n'
 '- Isolation: Each service runs in its own container, i

2024-04-27 13:21:45,095 [MainThread  ] [INFO ]  Done writing Article: # Increase Development Productivity with Docker Compose and Local Docker Labs

## Introduction to Docker Compose

Docker Compose is a tool for defining and running multi-container Docker applications.


Handling conversation:
	{'context': 'Increase development productivity by using Docker compose and local docker labs', 'interview_config': None, 'editor': {'name': 'John Doe', 'role': 'Lead Editor', 'affiliation': 'University of ABC', 'description': 'Specializes in environmental science.'}, 'messages': [], 'references': {}}
Handling conversation:
	{'context': 'Increase development productivity by using Docker compose and local docker labs', 'interview_config': None, 'editor': {'name': 'Jane Smith', 'role': 'Editor', 'affiliation': 'Company XYZ', 'description': 'Focuses on technology trends.'}, 'messages': [], 'references': {}}
Saving data to interviews1.json

{'topic': 'Increase development productivity by using Docker compose and local docker labs', 'outline': {'page_title': 'Increase Development Productivity with Docker Compose and Local Docker Labs', 'sections': [{'section_title': 'Introduction to Docker Compose', 'description': 'An overview of Docker Compose, a tool for defining an

Outline(page_title='Increase Development Productivity with Docker Compose and Local Docker Labs', sections=[Section(section_title='Introduction to Docker Compose', description='An overview of Docker Compose, a tool for defining and running multi-container Docker applications.', subsections=[Subsection(subsection_title='What is Docker Compose?', description='Definition of Docker Compose and its key features for simplifying container orchestration.'), Subsection(subsection_title='Advantages of Docker Compose', description='Exploration of the benefits of using Docker Compose for development, testing, and deployment processes.')]), Section(section_title='Setting Up Local Docker Labs', description='Guidance on establishing and managing local Docker labs to boost productivity and streamline development workflows.', subsections=[Subsection(subsection_title='Creating a Local Docker Lab', description='Step-by-step instructions on setting up a local Docker lab environment for practical experimen

## Reload from JSON


In [5]:
# read Interviews from interviews.json
with open("interviews.json", "r") as f:
    interviews = json.load(f)
    i = Interviews.from_dict(interviews)
    i.interview_config = interview_config
    for c in i.conversations or []:
        c.interview_config = interview_config

# i
i.wiki_sections

Interviews.from_dict: data is an instance of dict
InterviewState.from_dict: data is an instance of dict
InterviewState.from_dict: data is an instance of dict


[{'section_title': 'Introduction to Docker Compose',
  'content': 'Docker Compose is a tool for defining and running multi-container Docker applications. It simplifies the process of managing and orchestrating multiple Docker containers by providing a declarative way to specify the services, networks, and volumes that make up an application.\n\nDocker Compose allows developers to define the entire application stack in a single configuration file, making it easier to share, version, and deploy the application consistently across different environments. This helps to ensure that the development, testing, and production environments are as similar as possible, reducing the chances of encountering "it works on my machine" issues.\n\nSome of the key advantages of using Docker Compose include:\n\n- **Simplifying Container Orchestration**: Docker Compose abstracts away the complex task of managing and orchestrating multiple containers, making it easier to set up and manage multi-container app

In [6]:
# g1y = await node_generate_sections(i)
# g1y
# x
# i.conversations

In [7]:
# import pprint

# from langchain_community.utilities import SearxSearchWrapper

# search = SearxSearchWrapper(searx_host="http://192.168.68.84:8900")
# pprint.pprint(search.results("Docker compose", num_results=5, engines="arxiv")) # search.run("Docker compose")

In [8]:
i1 = await node_write_article(i)
# i1

2024-04-27 13:21:45,365 [MainThread  ] [INFO ]  Writing article for topic: Increase development productivity by using Docker compose and local docker labs


Interviews.from_dict: data is an instance of Interviews
Notes has reached token limit: 1044 > 1000
SectionTitle: Introduction to Docker Compose, Notes: Title: 

Source: https://blog.devops.dev/orchestrating-multi-container-applications-with-docker-compose-and-swarm-feba6624a27e

Summary: 

Docker Compose: Defining Multi-Container Applications. Docker
Notes has reached token limit: 1002 > 1000
SectionTitle: Setting Up Local Docker Labs, Notes: Title: 

Source: https://medium.com/@rahulbagai/navigating-docker-images-and-local-development-insights-and-strategies-8a6449de9a31

Summary: 

When integrating Docker into your development workflow o
Notes has reached token limit: 1059 > 1000
SectionTitle: Improving Development Productivity, Notes: Title: 

Source: https://dev.to/theramoliya/docker-utilize-docker-compose-for-local-development-environments-17di

Summary: 

By using Docker Compose, you can simplify, providing all the services and 
('### What is Docker Compose?\n'
 '\n'
 'Docker Com

2024-04-27 13:21:58,875 [MainThread  ] [INFO ]  Done writing Article: # Increase Development Productivity with Docker Compose and Local Docker Labs

## Introduction to Docker Compose

Docker Compose is a tool for defining and running multi-container Docker applications.


Handling conversation:
	InterviewState(context='Increase development productivity by using Docker compose and local docker labs', interview_config=InterviewConfig(long_llm=ChatAnthropic(model='claude-3-haiku-20240307', anthropic_api_key=SecretStr('**********'), _client=<anthropic.Anthropic object at 0x116836680>, _async_client=<anthropic.AsyncAnthropic object at 0x116836f20>), fast_llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x1166c8e80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x1166ca140>, openai_api_key=SecretStr('**********'), openai_proxy=''), max_conversations=3, max_reference_length=10000, tags_to_extract=['p', 'h1', 'h2', 'h3'], embeddings=GPT4AllEmbeddings(client=<gpt4all.gpt4all.Embed4All object at 0x1166ca1d0>), vectorstore_dir='./data/storm/vectorstore/', vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x1041d69e0>, interview_graph=<storm.StormInterviewGraph1 object at 0x1646c71c0>, 

In [9]:
# pprint.pprint(i1["article"])

from IPython.display import Markdown


article = i1["article"]


# We will down-header the sections to create less confusion in this notebook
Markdown(article.replace("\n#", "\n##"))

# Increase Development Productivity with Docker Compose and Local Docker Labs

### Introduction to Docker Compose

Docker Compose is a tool for defining and running multi-container Docker applications. It simplifies the process of managing and orchestrating multiple Docker containers by providing a declarative way to specify the services, networks, and volumes that make up an application.

#### What is Docker Compose?

Docker Compose allows developers to define the entire application stack in a single configuration file, making it easier to share, version, and deploy the application consistently across different environments. This helps to ensure that the development, testing, and production environments are as similar as possible, reducing the chances of encountering "it works on my machine" issues.

#### Advantages of Docker Compose

Some of the key advantages of using Docker Compose include:

- **Simplifying Container Orchestration**: Docker Compose abstracts away the complex task of managing and orchestrating multiple containers, making it easier to set up and manage multi-container applications.

- **Improving Consistency and Reliability**: By defining the entire application stack in a single configuration file, Docker Compose ensures that the development, testing, and production environments are consistent, reducing the chances of encountering environment-specific issues.

- **Boosting Development Productivity**: Docker Compose allows developers to quickly spin up and tear down development environments, enabling faster iteration and experimentation.

- **Enhancing Collaboration**: The declarative nature of Docker Compose configuration files makes it easier for team members to understand and collaborate on the application's architecture and deployment.

### Setting Up Local Docker Labs

Guidance on establishing and managing local Docker labs to boost productivity and streamline development workflows. Docker Compose is a tool that helps create and run multi-container Docker applications, simplifying the process of setting up and managing a local development environment. By using Docker Compose, developers can ensure consistency across team members, isolate dependencies, and boost overall productivity.

#### Creating a Local Docker Lab

To set up a local Docker lab, follow these steps:

1. Install Docker and Docker Compose on your development machine.
2. Create a `docker-compose.yml` file that defines the services, networks, and volumes for your application.
3. Start the local Docker lab by running `docker-compose up` in the directory containing the `docker-compose.yml` file.
4. Access and interact with the running containers and services in your local Docker lab.

#### Best Practices for Local Docker Labs

Here are some best practices for optimizing your local Docker labs:

- **Maintain Consistency**: Ensure that your local Docker lab closely mirrors your production environment to reduce the chances of encountering deployment issues.
- **Leverage Volumes**: Use Docker volumes to persist data and share files between containers, making it easier to work with and test your application.
- **Automate Workflows**: Integrate your local Docker lab with your development workflow, such as by including it in your CI/CD pipeline.
- **Encourage Collaboration**: Share your Docker Compose configuration with team members to promote collaboration and consistency across the development environment.

### Improving Development Productivity

Utilizing Docker Compose and local Docker labs to accelerate development processes and improve overall productivity.

#### Integration with Development Workflow

To integrate Docker Compose and local Docker labs into your development workflow, consider the following strategies:

- **Automated Provisioning**: Set up scripts or pipelines to automatically provision and tear down local Docker labs as needed, streamlining the development process.
- **Continuous Integration**: Incorporate your Docker Compose configuration into your CI/CD pipeline, ensuring consistent and reliable environments across different stages of the development lifecycle.
- **Version Control**: Store your Docker Compose configuration files in your version control system, making it easier to share, collaborate, and maintain the application's infrastructure.

#### Real-world Use Cases

Organizations have successfully leveraged Docker Compose and local Docker labs to achieve significant productivity gains and operational efficiencies. Here are a few real-world examples:

1. **Streamlining Development Workflows**: A software development team used Docker Compose to set up local development environments for each team member, reducing the time and effort required to onboard new developers and ensuring consistent development environments.

2. **Improving Collaboration and Experimentation**: A research team utilized local Docker labs to quickly spin up and tear down development environments, enabling faster prototyping and experimentation with different software stacks and configurations.

3. **Enhancing Deployment Reliability**: A DevOps team integrated their Docker Compose configuration into their CI/CD pipeline, ensuring that the development, staging, and production environments were consistent, and reducing the number of deployment-related issues.

By adopting Docker Compose and local Docker labs, organizations have reported increased efficiency, faster development

In [10]:
x

NameError: name 'x' is not defined

In [ ]:
# from langchain_core.vectorstores import VectorStore



# print(get_notes_from_vectorstore("What is the best way to increase development productivity by using Docker compose and local docker labs", vectorstore, ))

In [ ]:
# def get_wikipedia_retriever(k: int = 2, content_chars_max: int = 5000) -> WikipediaRetriever:
#     return WikipediaRetriever(load_all_available_meta=True, top_k_results=k, doc_content_chars_max=content_chars_max)

# wr = get_wikipedia_retriever()
# docs = wr.invoke("Fornite")

# print(f"Number of docs: {len(docs)}")
# for doc in docs:
#     print(f"==============\nLength: {len(doc.page_content)}")
#     pprint.pprint(doc.page_content[:300])
    
#     print(f"Metdata: {doc.metadata.keys()}")
#     print(f"Source: {doc.metadata['source']}")
#     print(f"URL: {doc.metadata['page_url']}")
#     print(f"Title: {doc.metadata['title']}")
#     print(f"Categories: {doc.metadata['categories']}")
#     print(f"Summary: {doc.metadata['summary']}")
#     print(f"References: {doc.metadata['references']}")
#     print(f"Related Titles: {doc.metadata['related_titles']}")
#     print(f"Sections: {doc.metadata['sections']}")

In [ ]:
## Generate Article

# await search_engine.ainvoke("Docker compose")

#### Generate Sections

Now you can generate the sections using the indexed docs.


#### Generate final article

Now we can rewrite the draft to appropriately group all the citations and maintain a consistent voice.


## Final Flow

Now it's time to string everything together. We will have 6 main stages in sequence:
.

1. Generate the initial outline + perspectives
2. Batch converse with each perspective to expand the content for the article
3. Refine the outline based on the conversations
4. Index the reference docs from the conversations
5. Write the individual sections of the article
6. Write the final wiki

The state tracks the outputs of each stage.


#### Create the graph


## Render the Wiki

Now we can render the final wiki page!
